In [1]:
#limpio la memoria
rm(list=ls())
gc()

#cargo librerias
library("data.table")
library("ROCR")
library("rpart")
library("rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604293,32.3,1291145,69,1291145,69.0
Vcells,1086160,8.3,8388608,64,1633205,12.5


Loading required package: ggplot2

Loading required package: lattice



In [2]:
setwd("~/lab1/")

#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")

#Formato para submit en Kaggle
#karchivo_score      <-  "../input/uamds2020ldi1f1/Sample_201910_Fase_I.txt"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "02-RPART"

#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000

#Establezco semilla aleatoria
set.seed(1)



In [3]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar

train_rows <- createDataPartition(dtrain$clase, p = .66, list = FALSE)

#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
dtest <- dtrain[-train_rows,]
dtrain <- dtrain[train_rows,]

dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [4]:
#Funcion que calcula la ganancia para una prediccion y valores reales
fmetrica_ganancia_rpart  = function( probs, clases, pclase_valor_positivo )
{
 
  return(  sum(    (probs > 1/40 ) * 
                   ifelse( clases== pclase_valor_positivo, kTPGain, kFPGain )   
              )
         )
}

In [5]:
# Funcion que entrena el arbol y luego determina la ganancia sobre el set de test
modelo_rpart_ganancia = function( ptrain, ptest, pmaxdepth, pminbucket, pminsplit, pcp )
{

  modelo   <-  rpart("clase_ternaria ~ .",   data = ptrain,  xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)

  #aplico el modelo a datos nuevos
  testing_prediccion  <- predict( modelo, ptest)[, "BAJA+2"]
    
  return(fmetrica_ganancia_rpart(testing_prediccion,ptest[,..kclase_nomcampo],"BAJA+2"))
  
}

In [6]:
#Cuenta los modelos probados
linea <- 1

#Establece una ganancia inicial muy baja a superar
max_gan <- -100000000

#minsplit= minimo de split para seguir particionando el arbol.
#maxdepth= maxima profundidad del arbol
#minbucket = minimo de obs por clase.
#CP= que tan distintos son los hijos al padr. mientras mayor mas diferenctes seran
#For anidados para grid search 

#for( vmaxdepth in  c(4,8,14,16) ) #max Depth
for( vmaxdepth in  c(14) ) #max Depth
for( vcp in c(-0.0478)) #Complexity Factor
#for( vcp in c( -1)) #Complexity Factor    
for( vminsplit in  c(1397 ) ) #Min Split
#for( vminsplit in  c(10, 15) ) #Min Split
for( vminbucket  in  unique( as.integer(c(6 )) ) ) #Min Bucket
{ 

    #Muestra los parametros a testear
    message(paste('Intentando',vcp,vminsplit,vminbucket,vmaxdepth))
    #Llama a la funcion que entrena el modelo y calcula la ganancia para esos parametros
    res <- modelo_rpart_ganancia( dtrain, dtest, 
                                  pmaxdepth=vmaxdepth, pminbucket=vminbucket, pminsplit=vminsplit, pcp=vcp )

  
    #Verifica si se supero la mejor ganancia hasta el momento. En caso de haberla superado graba los hiperparametros nuevos
    if (res>max_gan) {
      max_gan <- res
      message(paste("Nuevo Maximo: ", res*3, vcp,vminsplit,vminbucket,vmaxdepth))
      pcp<-vcp
      pminsplit<-vminsplit
      pminbucket<-vminbucket
      pmaxdepth<-vmaxdepth
  }
  #Muestra al ganancia de la iteracion
  message(paste('Ganancia:',res*3,'Ganancia Maxima',max_gan*3))
    
  linea <- linea+1


}

Intentando -0.0478 1397 6 14

Nuevo Maximo:  17328000 -0.0478 1397 6 14

Ganancia: 17328000 Ganancia Maxima 17328000



In [7]:
#Entrena el modelo con el dataset completo
t0       <-  Sys.time()
modelo   <-  rpart("clase_ternaria ~ .",   data = dataset,   xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)
t1       <-  Sys.time()

tcorrida <-  as.numeric( t1 - t0, units = "secs")
print( tcorrida)

[1] 29.18908


In [9]:
#Detalles del modelo
modelo

n= 324242 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

    1) root 324242 161342  (5.024025e-01 2.047853e-03 2.621499e-03 4.929281e-01)  
      2) foto_mes>=202102 162900      0  (1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00) *
      3) foto_mes< 202102 161342   1514 CONTINUA (0.000000e+00 4.115481e-03 5.268312e-03 9.906162e-01)  
        6) ctrx_quarter< 13.5 8400    711 CONTINUA (0.000000e+00 3.476190e-02 4.988095e-02 9.153571e-01)  
         12) mcuentas_saldo< -1256.1 2928    416 CONTINUA (0.000000e+00 6.181694e-02 8.025956e-02 8.579235e-01)  
           24) cprestamos_personales< 1.5 2328    396 CONTINUA (0.000000e+00 7.431271e-02 9.579038e-02 8.298969e-01)  
             48) cdescubierto_preacordado< 0.5 321    116 CONTINUA (0.000000e+00 1.713396e-01 1.900312e-01 6.386293e-01) *
             49) cdescubierto_preacordado>=0.5 2007    280 CONTINUA (0.000000e+00 5.879422e-02 8.071749e-02 8.604883e-01)  
               98) mpasivos_margen< 8.05 1339   

In [11]:
#Leo datos a estimar
dataset_pred <- fread(karchivo_predecir)


#Genero la clase
#dataset_pred[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
#dataset_pred[ ,  (kcampos_a_borrar) := NULL    ] 

prediccion  <- predict( modelo, dataset_pred ) #aplico el modelo

ERROR: Error in fread(karchivo_predecir): object 'karchivo_predecir' not found


In [12]:
dataset_pred[ , prob_baja := prediccion]
dataset_pred[ , Predicted  := as.numeric(prob_baja > 0.025) ]

entrega  <- dataset_pred[   , list(numero_de_cliente, Predicted) ] #genero la salida

#genero el archivo para Kaggle
fwrite( entrega, file="./salida_rpart_grid.csv", sep="," )

ERROR: Error in eval(expr, envir, enclos): object 'dataset_pred' not found
